In [13]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import urllib.parse
from sqlalchemy import create_engine
from geopy.distance import geodesic
import os
from tqdm import tqdm
tqdm.pandas()
df_tienda = pd.read_csv("../Data/DIM_TIENDA.csv")
# Sólo quedarse con las columnas "TIENDA_ID, LATITUD_NUM, LONGITUD_NUM"
df_tiendas = df_tienda[["TIENDA_ID", "LATITUD_NUM", "LONGITUD_NUM"]]

In [14]:
user = 'root'
raw_password = os.getenv("SQL_PASSWORD")
password = password = urllib.parse.quote_plus(raw_password)
host = 'localhost'
database = 'cat_ageeml'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# Carga las tablas
df_localidades = pd.read_sql("SELECT id, nombre, lat, lng FROM localidades WHERE lat IS NOT NULL AND lng IS NOT NULL AND activo = 1", engine)

In [15]:
# --- CONVERTIR COORDENADAS A RADIANES ---
localidades_coords_rad = np.radians(df_localidades[['lat', 'lng']].values)
tiendas_coords_rad = np.radians(df_tiendas[['LATITUD_NUM', 'LONGITUD_NUM']].values)

# --- CREAR BALL TREE CON MÉTRICA HAVERSINE ---
tree = BallTree(localidades_coords_rad, metric='haversine')

# --- CONSULTAR VECINO MÁS CERCANO PARA CADA TIENDA ---
# El resultado está en radianes, multiplicamos por radio de la Tierra para km
distancias, indices = tree.query(tiendas_coords_rad, k=1)
distancias_km = distancias.flatten() * 6371  # 6371 km es el radio de la Tierra

# --- ASIGNAR RESULTADOS ---
df_tiendas['localidad_id'] = df_localidades.iloc[indices.flatten()].reset_index(drop=True)['id']
df_tiendas['localidad_nombre'] = df_localidades.iloc[indices.flatten()].reset_index(drop=True)['nombre']
df_tiendas['distancia_km'] = distancias_km

# --- GUARDAR RESULTADO ---
df_tiendas.to_csv('tiendas_con_localidad_balltree.csv', index=False)

C:\Users\aesca\AppData\Local\Temp\ipykernel_33732\2309382445.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tiendas['localidad_id'] = df_localidades.iloc[indices.flatten()].reset_index(drop=True)['id']
C:\Users\aesca\AppData\Local\Temp\ipykernel_33732\2309382445.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tiendas['localidad_nombre'] = df_localidades.iloc[indices.flatten()].reset_index(drop=True)['nombre']
C:\Users\aesca\AppData\Local\Temp\ipykernel_33732\2309382445.py:16: SettingWithCop

In [1]:
# # --- APLICAR ---
# df_tiendas['localidad_id'] = df_tiendas.progress_apply(
#     lambda row: encontrar_localidad_mas_cercana_opt(row['LATITUD_NUM'], row['LONGITUD_NUM']), axis=1
# )

# # --- OPCIONAL: Añadir nombre
# df_tiendas['localidad_nombre'] = df_tiendas['localidad_id'].map(
#     df_localidades.set_index('id')['nombre']
# )

# # --- GUARDAR RESULTADO ---
# df_tiendas.to_csv('tiendas_con_localidad.csv', index=False)